# Long-Running Memory Demo

This notebook demonstrates **Session Management** and **Conversation History** features of the .NET Agent Framework backend API.

## What You'll Learn
- How to create and manage chat sessions
- How conversation history is maintained across messages
- Working with session IDs for persistent conversations
- Understanding context retention in multi-turn conversations
- Best practices for session management

## Prerequisites
Make sure the backend API is running on `http://localhost:8000`

## Setup - Install Required Packages

In [ ]:
#r "nuget: System.Net.Http.Json"
#r "nuget: System.Text.Json"

using System.Net.Http;
using System.Net.Http.Json;
using System.Text.Json;
using System.Text.Json.Serialization;

## Configure API Connection

In [ ]:
// API Configuration
var apiBaseUrl = "http://localhost:8000";
var httpClient = new HttpClient { BaseAddress = new Uri(apiBaseUrl) };
httpClient.Timeout = TimeSpan.FromMinutes(2);

Console.WriteLine($"✅ Connected to API: {apiBaseUrl}");
Console.WriteLine($"🧠 Memory & Session Management Demo\n");

## Step 1: Create a New Session (First Message)

When you don't provide a session_id, the API creates a new session automatically.

In [ ]:
// Send first message without session ID (creates new session)
var firstMessage = new
{
    message = "My name is Alice and I work as a software engineer at TechCorp.",
    agents = new[] { "generic_agent" },
    format = "user_friendly"
};

Console.WriteLine("📤 Sending FIRST message (creating new session)...");
Console.WriteLine($"Message: {firstMessage.message}\n");

var response1 = await httpClient.PostAsJsonAsync("/chat", firstMessage);
response1.EnsureSuccessStatusCode();

var result1 = await response1.Content.ReadFromJsonAsync<JsonElement>();

// Extract and save the session ID
var sessionId = result1.GetProperty("session_id").GetString();

Console.WriteLine("💬 Response:");
Console.WriteLine("============");
Console.WriteLine($"Session ID: {sessionId}");
Console.WriteLine($"Agent: {result1.GetProperty("agent").GetString()}");
Console.WriteLine($"\nContent: {result1.GetProperty("content").GetString()}");

// Check conversation length
if (result1.TryGetProperty("metadata", out var meta1) && 
    meta1.TryGetProperty("conversation_length", out var convLen1))
{
    Console.WriteLine($"\n📊 Conversation Length: {convLen1.GetInt32()} messages");
}

Console.WriteLine($"\n✅ Session created! Session ID: {sessionId}");

## Step 2: Continue Conversation (Second Message)

Use the session_id from the previous message to continue the conversation.

In [ ]:
// Send second message with session ID
var secondMessage = new
{
    message = "What's my name and where do I work?",
    session_id = sessionId,  // Use the session ID from previous message
    agents = new[] { "generic_agent" },
    format = "user_friendly"
};

Console.WriteLine("📤 Sending SECOND message (continuing session)...");
Console.WriteLine($"Message: {secondMessage.message}");
Console.WriteLine($"Session ID: {secondMessage.session_id}\n");

var response2 = await httpClient.PostAsJsonAsync("/chat", secondMessage);
response2.EnsureSuccessStatusCode();

var result2 = await response2.Content.ReadFromJsonAsync<JsonElement>();

Console.WriteLine("💬 Response:");
Console.WriteLine("============");
Console.WriteLine($"Session ID: {result2.GetProperty("session_id").GetString()}");
Console.WriteLine($"\nContent: {result2.GetProperty("content").GetString()}");

// Check conversation length
if (result2.TryGetProperty("metadata", out var meta2) && 
    meta2.TryGetProperty("conversation_length", out var convLen2))
{
    Console.WriteLine($"\n📊 Conversation Length: {convLen2.GetInt32()} messages");
}

Console.WriteLine($"\n✅ The agent remembered information from the first message!");

## Step 3: Multi-Turn Conversation

Continue the conversation for several turns to demonstrate context retention.

In [ ]:
// Continue with multiple turns
var conversationTurns = new[]
{
    "I'm working on a new project using Azure AI services.",
    "Can you remind me what project I'm working on?",
    "What technologies am I using in my project?",
    "Summarize everything you know about me."
};

Console.WriteLine("🔄 Multi-Turn Conversation");
Console.WriteLine("==========================\n");

int turnNumber = 3; // We already had 2 turns

foreach (var turn in conversationTurns)
{
    Console.WriteLine($"\n[Turn {turnNumber}]");
    Console.WriteLine("-".PadRight(70, '-'));
    Console.WriteLine($"👤 User: {turn}");
    
    var turnMessage = new
    {
        message = turn,
        session_id = sessionId,
        agents = new[] { "generic_agent" },
        format = "user_friendly"
    };
    
    try
    {
        var turnResponse = await httpClient.PostAsJsonAsync("/chat", turnMessage);
        turnResponse.EnsureSuccessStatusCode();
        
        var turnResult = await turnResponse.Content.ReadFromJsonAsync<JsonElement>();
        var content = turnResult.GetProperty("content").GetString();
        
        Console.WriteLine($"🤖 Agent: {content}");
        
        // Show conversation length
        if (turnResult.TryGetProperty("metadata", out var meta) && 
            meta.TryGetProperty("conversation_length", out var convLen))
        {
            Console.WriteLine($"📊 Conversation length: {convLen.GetInt32()} messages");
        }
        
        turnNumber++;
        
        // Brief pause between turns
        await Task.Delay(500);
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error: {ex.Message}");
    }
}

Console.WriteLine($"\n\n✅ Completed {turnNumber} turns in the same conversation!");
Console.WriteLine($"   Session ID: {sessionId}");

## Step 4: Start a New Session (Fresh Context)

When you start a new session, the agent won't remember previous conversations.

In [ ]:
// Start a completely new session
Console.WriteLine("🆕 Starting NEW Session (Fresh Context)");
Console.WriteLine("=======================================");
Console.WriteLine($"Previous Session ID: {sessionId}\n");

var newSessionMessage = new
{
    message = "What's my name?",  // This was mentioned in the old session
    // No session_id provided = new session
    agents = new[] { "generic_agent" },
    format = "user_friendly"
};

Console.WriteLine($"📤 Sending: {newSessionMessage.message}");
Console.WriteLine($"   (Without session ID - creates new session)\n");

var newSessionResponse = await httpClient.PostAsJsonAsync("/chat", newSessionMessage);
newSessionResponse.EnsureSuccessStatusCode();

var newSessionResult = await newSessionResponse.Content.ReadFromJsonAsync<JsonElement>();
var newSessionId = newSessionResult.GetProperty("session_id").GetString();

Console.WriteLine("💬 Response:");
Console.WriteLine("============");
Console.WriteLine($"New Session ID: {newSessionId}");
Console.WriteLine($"\nContent: {newSessionResult.GetProperty("content").GetString()}");

Console.WriteLine($"\n📊 Session Comparison:");
Console.WriteLine($"   Old Session: {sessionId}");
Console.WriteLine($"   New Session: {newSessionId}");
Console.WriteLine($"   Sessions are different: {sessionId != newSessionId}");
Console.WriteLine($"\n✅ The agent doesn't remember your name because it's a new session!");

## Step 5: Session Management with Multiple Agents

Session memory works across multiple agents in group chat scenarios.

In [ ]:
// Create a new session with multiple agents
Console.WriteLine("👥 Multi-Agent Session with Memory");
Console.WriteLine("===================================\n");

// First message - establish context
var multiAgentMsg1 = new
{
    message = "I'm planning a cloud migration project for a retail company with 500 employees.",
    agents = new[] { "generic_agent", "knowledge_finder" },
    max_turns = 3,
    format = "user_friendly"
};

Console.WriteLine($"[Turn 1] 👤 User: {multiAgentMsg1.message}\n");

var multiResp1 = await httpClient.PostAsJsonAsync("/chat", multiAgentMsg1);
var multiResult1 = await multiResp1.Content.ReadFromJsonAsync<JsonElement>();
var multiSessionId = multiResult1.GetProperty("session_id").GetString();

Console.WriteLine($"🤖 Response: {multiResult1.GetProperty("content").GetString().Substring(0, 200)}...\n");
Console.WriteLine($"Session ID: {multiSessionId}");

// Second message - reference previous context
await Task.Delay(1000);

var multiAgentMsg2 = new
{
    message = "What are the key considerations for the project I mentioned?",
    session_id = multiSessionId,
    agents = new[] { "generic_agent", "knowledge_finder" },
    max_turns = 3,
    format = "user_friendly"
};

Console.WriteLine($"\n[Turn 2] 👤 User: {multiAgentMsg2.message}\n");

var multiResp2 = await httpClient.PostAsJsonAsync("/chat", multiAgentMsg2);
var multiResult2 = await multiResp2.Content.ReadFromJsonAsync<JsonElement>();

Console.WriteLine($"🤖 Response: {multiResult2.GetProperty("content").GetString().Substring(0, 200)}...");

// Show metadata
if (multiResult2.TryGetProperty("metadata", out var multiMeta))
{
    Console.WriteLine($"\n📊 Session Metadata:");
    Console.WriteLine($"   Agent Count: {multiMeta.GetProperty("agent_count").GetInt32()}");
    Console.WriteLine($"   Total Turns: {multiMeta.GetProperty("total_turns").GetInt32()}");
    
    if (multiMeta.TryGetProperty("conversation_length", out var multiConvLen))
    {
        Console.WriteLine($"   Conversation Length: {multiConvLen.GetInt32()} messages");
    }
    
    if (multiMeta.TryGetProperty("contributing_agents", out var contrib))
    {
        var agents = string.Join(", ", contrib.EnumerateArray().Select(a => a.GetString()));
        Console.WriteLine($"   Contributing Agents: {agents}");
    }
}

Console.WriteLine($"\n✅ Multiple agents share the same conversation history!");

## Step 6: Best Practices for Session Management

In [ ]:
Console.WriteLine("📚 Session Management Best Practices");
Console.WriteLine("====================================\n");

Console.WriteLine("✅ DO:");
Console.WriteLine("   1. Store session IDs on the client side");
Console.WriteLine("   2. Pass session_id for follow-up messages in the same conversation");
Console.WriteLine("   3. Create new sessions for unrelated conversations");
Console.WriteLine("   4. Use session IDs to maintain context across multiple turns");
Console.WriteLine("   5. Allow users to start fresh conversations (new sessions)");
Console.WriteLine();

Console.WriteLine("❌ DON'T:");
Console.WriteLine("   1. Reuse session IDs across different users");
Console.WriteLine("   2. Store sensitive information assuming permanent retention");
Console.WriteLine("   3. Expect sessions to last forever (implement timeout logic)");
Console.WriteLine("   4. Mix unrelated conversations in the same session");
Console.WriteLine();

Console.WriteLine("💡 Tips:");
Console.WriteLine("   • Session IDs are GUIDs (Globally Unique Identifiers)");
Console.WriteLine("   • Conversation history helps agents provide better, context-aware responses");
Console.WriteLine("   • Multi-agent conversations share the same session memory");
Console.WriteLine("   • Session data includes user messages and agent responses");
Console.WriteLine();

Console.WriteLine("🔍 Example Session Flow:");
Console.WriteLine("   User sends message (no session_id) → Server creates new session");
Console.WriteLine("   Server returns session_id → Client stores it");
Console.WriteLine("   User sends follow-up (with session_id) → Server retrieves history");
Console.WriteLine("   Agent responds with context → Conversation continues");

## Step 7: Session ID Structure and Management

In [ ]:
// Demonstrate session ID structure
Console.WriteLine("🔑 Session ID Information");
Console.WriteLine("=========================");
Console.WriteLine();

// Show the session IDs we created
Console.WriteLine("Session IDs created in this notebook:");
Console.WriteLine($"   First Session:  {sessionId}");
Console.WriteLine($"   Second Session: {newSessionId}");
Console.WriteLine($"   Multi-Agent:    {multiSessionId}");
Console.WriteLine();

// Validate GUID format
bool isValidGuid1 = Guid.TryParse(sessionId, out var guid1);
bool isValidGuid2 = Guid.TryParse(newSessionId, out var guid2);
bool isValidGuid3 = Guid.TryParse(multiSessionId, out var guid3);

Console.WriteLine("✅ All session IDs are valid GUIDs:");
Console.WriteLine($"   Session 1: {isValidGuid1}");
Console.WriteLine($"   Session 2: {isValidGuid2}");
Console.WriteLine($"   Session 3: {isValidGuid3}");
Console.WriteLine();

Console.WriteLine("📊 Session ID Properties:");
Console.WriteLine($"   Format: GUID (RFC 4122)");
Console.WriteLine($"   Length: 36 characters (including hyphens)");
Console.WriteLine($"   Example: {sessionId}");
Console.WriteLine($"   Uniqueness: Globally unique (collision probability ≈ 0)");
Console.WriteLine();

Console.WriteLine("💾 Recommended Client-Side Storage:");
Console.WriteLine("   • Web: localStorage or sessionStorage");
Console.WriteLine("   • Mobile: Secure storage or app state");
Console.WriteLine("   • Desktop: Application settings or cache");
Console.WriteLine();

Console.WriteLine("⏰ Session Lifecycle:");
Console.WriteLine("   1. Created: When first message is sent without session_id");
Console.WriteLine("   2. Active: While messages are being exchanged");
Console.WriteLine("   3. Stored: Session history maintained on server");
Console.WriteLine("   4. Retrieved: When session_id is provided in subsequent messages");

## Summary

In this notebook, you learned:
- ✅ How to create and manage chat sessions
- ✅ How conversation history is maintained across messages
- ✅ The importance of session IDs for context retention
- ✅ Multi-turn conversations with context awareness
- ✅ How to start fresh conversations with new sessions
- ✅ Session management with multiple agents
- ✅ Best practices for session handling

## Key Concepts

### Session ID
- Unique identifier (GUID) for a conversation
- Generated automatically if not provided
- Must be passed to continue the same conversation

### Conversation History
- Stores all messages in a session (user + agent)
- Enables context-aware responses
- Shared across all agents in group chat

### Context Retention
- Agents remember previous messages in the same session
- Can reference information from earlier in the conversation
- New sessions start with empty context

## Real-World Use Cases
1. **Customer Support**: Maintain conversation context across multiple interactions
2. **Task Assistance**: Remember user preferences and progress
3. **Learning Systems**: Track educational progress and adapt responses
4. **Complex Workflows**: Break down tasks into multiple steps with context

## Next Steps
- Review the **Single Agent Demo** for basic agent interaction
- Explore the **Multiple Agents Demo** for group chat scenarios
- Check the **Content Safety Demo** for content moderation features